# Lab 5


Matrix Representation: In this lab you will be creating a simple linear algebra system. In memory, we will represent matrices as nested python lists as we have done in lecture. In the exercises below, you are required to explicitly test every feature you implement, demonstrating it works.

1. Create a `matrix` class with the following properties:
    * It can be initialized in 2 ways:
        1. with arguments `n` and `m`, the size of the matrix. A newly instanciated matrix will contain all zeros.
        2. with a list of lists of values. Note that since we are using lists of lists to implement matrices, it is possible that not all rows have the same number of columns. Test explicitly that the matrix is properly specified.
    * Matrix instances `M` can be indexed with `M[i][j]` and `M[i,j]`.
    * Matrix assignment works in 2 ways:
        1. If `M_1` and `M_2` are `matrix` instances `M_1=M_2` sets the values of `M_1` to those of `M_2`, if they are the same size. Error otherwise.
        2. In example above `M_2` can be a list of lists of correct size.


2. Add the following methods:
    * `shape()`: returns a tuple `(n,m)` of the shape of the matrix.
    * `transpose()`: returns a new matrix instance which is the transpose of the matrix.
    * `row(n)` and `column(n)`: that return the nth row or column of the matrix M as a new appropriately shaped matrix object.
    * `to_list()`: which returns the matrix as a list of lists.
    *  `block(n_0,n_1,m_0,m_1)` that returns a smaller matrix located at the n_0 to n_1 columns and m_0 to m_1 rows. 
    * (Extra credit) Modify `__getitem__` implemented above to support slicing.
        

3. Write functions that create special matrices (note these are standalone functions, not member functions of your `matrix` class):
    * `constant(n,m,c)`: returns a `n` by `m` matrix filled with floats of value `c`.
    * `zeros(n,m)` and `ones(n,m)`: return `n` by `m` matrices filled with floats of value `0` and `1`, respectively.
    * `eye(n)`: returns the n by n identity matrix.

4. Add the following member functions to your class. Make sure to appropriately test the dimensions of the matrices to make sure the operations are correct.
    * `M.scalarmul(c)`: a matrix that is scalar product $cM$, where every element of $M$ is multiplied by $c$.
    * `M.add(N)`: adds two matrices $M$ and $N$. Don’t forget to test that the sizes of the matrices are compatible for this and all other operations.
    * `M.sub(N)`: subtracts two matrices $M$ and $N$.
    * `M.mat_mult(N)`: returns a matrix that is the matrix product of two matrices $M$ and $N$.
    * `M.element_mult(N)`: returns a matrix that is the element-wise product of two matrices $M$ and $N$.
    * `M.equals(N)`: returns true/false if $M==N$.

5. Overload python operators to appropriately use your functions in 4 and allow expressions like:
    * 2*M
    * M*2
    * M+N
    * M-N
    * M*N
    * M==N
    * M=N


6. Demonstrate the basic properties of matrices with your matrix class by creating two 2 by 2 example matrices using your Matrix class and illustrating the following:

$$
(AB)C=A(BC)
$$
$$
A(B+C)=AB+AC
$$
$$
AB\neq BA
$$
$$
AI=A
$$

In [1]:
class Matrix:
    def __init__(self, *args):
        # Initialize matrix with either (n, m) or list of lists
        if len(args) == 2 and isinstance(args[0], int) and isinstance(args[1], int):
            self.rows = args[0]
            self.cols = args[1]
            self.matrix = [[0 for _ in range(self.cols)] for _ in range(self.rows)]
        elif len(args) == 1 and isinstance(args[0], list):
            self.matrix = args[0]
            self.rows = len(self.matrix)
            self.cols = len(self.matrix[0]) if self.matrix else 0
            # Check if all rows have the same number of columns
            if not all(len(row) == self.cols for row in self.matrix):
                raise ValueError("Matrix rows do not have the same number of columns")
        else:
            raise ValueError("Invalid initialization parameters")
    
    def __getitem__(self, idx):
        if isinstance(idx, tuple):
            row, col = idx
            if isinstance(row, slice) or isinstance(col, slice):
                return Matrix([r[col] for r in self.matrix[row]])
            return self.matrix[row][col]
        elif isinstance(idx, int):
            return self.matrix[idx]
        else:
            raise TypeError("Invalid index type")

    def __setitem__(self, idx, value):
        if isinstance(idx, tuple):
            self.matrix[idx[0]][idx[1]] = value
        elif isinstance(idx, int):
            self.matrix[idx] = value
        else:
            raise TypeError("Invalid index type")

    def assign(self, other):
        if isinstance(other, Matrix):
            if self.rows != other.rows or self.cols != other.cols:
                raise ValueError("Matrices must be the same size for assignment")
            self.matrix = [row[:] for row in other.matrix]
        elif isinstance(other, list):
            if len(other) != self.rows or len(other[0]) != self.cols:
                raise ValueError("List of lists must match matrix dimensions")
            self.matrix = [row[:] for row in other]
        else:
            raise TypeError("Assignment only supports Matrix or list of lists")

    def shape(self):
        return self.rows, self.cols

    def transpose(self):
        transposed_matrix = [[self.matrix[j][i] for j in range(self.rows)] for i in range(self.cols)]
        return Matrix(transposed_matrix)

    def row(self, n):
        return Matrix([self.matrix[n]])

    def column(self, n):
        col = [[self.matrix[i][n]] for i in range(self.rows)]
        return Matrix(col)

    def to_list(self):
        return self.matrix

    def block(self, n_0, n_1, m_0, m_1):
        block_matrix = [row[m_0:m_1] for row in self.matrix[n_0:n_1]]
        return Matrix(block_matrix)

    # New member functions as required

    def scalarmul(self, c):
        """Scalar multiplication of matrix with a constant c."""
        return Matrix([[c * self.matrix[i][j] for j in range(self.cols)] for i in range(self.rows)])

    def add(self, N):
        """Adds two matrices M and N."""
        if self.shape() != N.shape():
            raise ValueError("Matrix dimensions must match for addition")
        return Matrix([[self.matrix[i][j] + N.matrix[i][j] for j in range(self.cols)] for i in range(self.rows)])

    def sub(self, N):
        """Subtracts matrix N from M."""
        if self.shape() != N.shape():
            raise ValueError("Matrix dimensions must match for subtraction")
        return Matrix([[self.matrix[i][j] - N.matrix[i][j] for j in range(self.cols)] for i in range(self.rows)])

    def mat_mult(self, N):
        """Returns the matrix product of two matrices M and N."""
        if self.cols != N.rows:
            raise ValueError("Matrix multiplication not possible, incompatible dimensions")
        return Matrix([[sum(self.matrix[i][k] * N.matrix[k][j] for k in range(self.cols)) for j in range(N.cols)] for i in range(self.rows)])

    def element_mult(self, N):
        """Element-wise multiplication of two matrices M and N."""
        if self.shape() != N.shape():
            raise ValueError("Matrix dimensions must match for element-wise multiplication")
        return Matrix([[self.matrix[i][j] * N.matrix[i][j] for j in range(self.cols)] for i in range(self.rows)])

    def equals(self, N):
        """Returns true if two matrices are equal."""
        return self.matrix == N.matrix

    # Overloading operators
    def __mul__(self, other):
        if isinstance(other, (int, float)):  # Scalar multiplication
            return self.scalarmul(other)
        elif isinstance(other, Matrix):  # Matrix multiplication
            return self.mat_mult(other)
        else:
            raise ValueError("Unsupported operand for multiplication")

    def __rmul__(self, other):
        return self.__mul__(other)

    def __add__(self, other):
        return self.add(other)

    def __sub__(self, other):
        return self.sub(other)

    def __eq__(self, other):
        return self.equals(other)

    def __repr__(self):
        return f"Matrix({self.matrix})"


# Helper function for creating identity matrix
def eye(n):
    identity_matrix = [[1.0 if i == j else 0.0 for j in range(n)] for i in range(n)]
    return Matrix(identity_matrix)

# Demonstration using the matrix class
A = Matrix([[1, 2], [3, 4]])
B = Matrix([[2, 0], [1, 2]])
C = Matrix([[0, 1], [1, 0]])
I = eye(2)

# (AB)C = A(BC)
left_side = (A * B) * C
right_side = A * (B * C)

# A(B+C) = AB + AC
distributive_left = A * (B + C)
distributive_right = (A * B) + (A * C)

# AB ≠ BA
commutative_check_1 = A * B
commutative_check_2 = B * A

# AI = A
identity_check = A * I

(left_side, right_side, distributive_left, distributive_right, commutative_check_1, commutative_check_2, identity_check)

(Matrix([[4, 4], [8, 10]]),
 Matrix([[4, 4], [8, 10]]),
 Matrix([[6, 5], [14, 11]]),
 Matrix([[6, 5], [14, 11]]),
 Matrix([[4, 4], [10, 8]]),
 Matrix([[2, 4], [7, 10]]),
 Matrix([[1.0, 2.0], [3.0, 4.0]]))

**Quiz 2:** Write a function make_deck that returns a list of all of the cards in a standard card deck. The return should be a list of tuples of pairs of suit and value. For example the 10 of Clubs would be  ('Clubs', 10) and Queen of Hearts would be ('Hearts', 'Queen'). Recall that a deck has 52 cards, divided into 4 suits (Clubs, Diamonds, Hearts, and Spades), and that each suit has 13 cards: 2 to 10, Jack, Queen, King, and Ace. Summit your solution with Lab 5.

In [3]:
def make_deck():
    suits = ['Clubs', 'Diamonds', 'Hearts', 'Spades']
    values = [2, 3, 4, 5, 6, 7, 8, 9, 10, 'Jack', 'Queen', 'King', 'Ace']
    
    deck = [(suit, value) for suit in suits for value in values]
    return deck

# Test the function
deck_of_cards = make_deck()
deck_of_cards[:52]

[('Clubs', 2),
 ('Clubs', 3),
 ('Clubs', 4),
 ('Clubs', 5),
 ('Clubs', 6),
 ('Clubs', 7),
 ('Clubs', 8),
 ('Clubs', 9),
 ('Clubs', 10),
 ('Clubs', 'Jack'),
 ('Clubs', 'Queen'),
 ('Clubs', 'King'),
 ('Clubs', 'Ace'),
 ('Diamonds', 2),
 ('Diamonds', 3),
 ('Diamonds', 4),
 ('Diamonds', 5),
 ('Diamonds', 6),
 ('Diamonds', 7),
 ('Diamonds', 8),
 ('Diamonds', 9),
 ('Diamonds', 10),
 ('Diamonds', 'Jack'),
 ('Diamonds', 'Queen'),
 ('Diamonds', 'King'),
 ('Diamonds', 'Ace'),
 ('Hearts', 2),
 ('Hearts', 3),
 ('Hearts', 4),
 ('Hearts', 5),
 ('Hearts', 6),
 ('Hearts', 7),
 ('Hearts', 8),
 ('Hearts', 9),
 ('Hearts', 10),
 ('Hearts', 'Jack'),
 ('Hearts', 'Queen'),
 ('Hearts', 'King'),
 ('Hearts', 'Ace'),
 ('Spades', 2),
 ('Spades', 3),
 ('Spades', 4),
 ('Spades', 5),
 ('Spades', 6),
 ('Spades', 7),
 ('Spades', 8),
 ('Spades', 9),
 ('Spades', 10),
 ('Spades', 'Jack'),
 ('Spades', 'Queen'),
 ('Spades', 'King'),
 ('Spades', 'Ace')]